In [1]:
import numpy as np
import pandas as pd
import os

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

# dataframe merge
 - SQL의 join처럼 특정한 column을 기준으로 병합
   - join 방식: how 파라미터를 통해 명시
     - inner: 기본값, 일치하는 값이 있는 경우 
     - left: left outer join
     - right: right outer join
     - outer: full outer join
     
 - pandas.merge 함수가 사용된다.

고객 데이터 테이블

In [2]:
customer = pd.DataFrame({'customer_id' : np.arange(6), 
                    'name' : ['철수'"", '영희', '길동', '영수', '수민', '동건'], 
                    '나이' : [40, 20, 21, 30, 31, 18]})

customer

,customer_id,name,나이
0,0,철수,40
1,1,영희,20
2,2,길동,21
3,3,영수,30
4,4,수민,31
5,5,동건,18


구매내역 데이터 테이블

In [3]:
orders = pd.DataFrame({'customer_id' : [1, 1, 2, 2, 2, 3, 3, 1, 4, 9], 
                    'item' : ['치약', '칫솔', '이어폰', '헤드셋', '수건', '생수', '수건', '치약', '생수', '케이스'], 
                    'quantity' : [1, 2, 1, 1, 3, 2, 2, 3, 2, 1]})
orders.head()

,customer_id,item,quantity
0,1,치약,1
1,1,칫솔,2
2,2,이어폰,1
3,2,헤드셋,1
4,2,수건,3


# on 
 - join 대상이 되는 column을 'on= 옵션'에 명시해준다.

## how=inner

In [4]:
# customer_id를 기준으로 병합
# how='inner' : 두 테이블간 '일치하는 값이 있는경우에만' 병합
# 여기서는 customer_id가 양 테이블에 공통으로 존재하는 컬럼

pd.merge(customer, orders, on='customer_id', how='inner')

,customer_id,name,나이,item,quantity
0,1,영희,20,치약,1
1,1,영희,20,칫솔,2
2,1,영희,20,치약,3
3,2,길동,21,이어폰,1
4,2,길동,21,헤드셋,1
5,2,길동,21,수건,3
6,3,영수,30,생수,2
7,3,영수,30,수건,2
8,4,수민,31,생수,2


## how = left

- left에 있는 customer 테이블에 있는 '동건'의 customer_id는 right에는 존재하지 않는다.
- how='left' 이기 때문에 '동건' 이라는 customer의 값은 살아 남지만, merge의 대상인 orders에는 대응하는 값이 없으므로 NaN 이된다.

In [5]:
pd.merge(customer, orders, on='customer_id', how='left')

,customer_id,name,나이,item,quantity
0,0,철수,40,NaN,NaN
1,1,영희,20,치약,1.0
2,1,영희,20,칫솔,2.0
3,1,영희,20,치약,3.0
4,2,길동,21,이어폰,1.0
5,2,길동,21,헤드셋,1.0
6,2,길동,21,수건,3.0
7,3,영수,30,생수,2.0
8,3,영수,30,수건,2.0
9,4,수민,31,생수,2.0


## how = right

- right에 있는 orders 테이블에 있는 '케이스' 구매 이력에 있는 'customer_id'는 left에는 존재하지 않는다.
- how='right' 이기 때문에 '케이스' 이라는 order의 값은 살아 남지만, merge의 대상인 customers에는 대응하는 값이 없으므로 NaN 이된다.

In [6]:
pd.merge(customer, orders, on='customer_id', how='right')

,customer_id,name,나이,item,quantity
0,1,영희,20.0,치약,1
1,1,영희,20.0,칫솔,2
2,2,길동,21.0,이어폰,1
3,2,길동,21.0,헤드셋,1
4,2,길동,21.0,수건,3
5,3,영수,30.0,생수,2
6,3,영수,30.0,수건,2
7,1,영희,20.0,치약,3
8,4,수민,31.0,생수,2
9,9,NaN,NaN,케이스,1


## how = outer

- 물리적으로 두 테이블을 '무조건' 합치는 것
- 일치하지 않는 값은 NaN 이 된다.

In [7]:
pd.merge(customer, orders, on='customer_id', how='outer')

,customer_id,name,나이,item,quantity
0,0,철수,40.0,NaN,NaN
1,1,영희,20.0,치약,1.0
2,1,영희,20.0,칫솔,2.0
3,1,영희,20.0,치약,3.0
4,2,길동,21.0,이어폰,1.0
5,2,길동,21.0,헤드셋,1.0
6,2,길동,21.0,수건,3.0
7,3,영수,30.0,생수,2.0
8,3,영수,30.0,수건,2.0
9,4,수민,31.0,생수,2.0


# index 기준으로 join하기

In [8]:
# 기존 데이터 테이블의 customer_id를 index로 설정
cust1 = customer.set_index('customer_id')
order1 = orders.set_index('customer_id')

In [9]:
cust1

,name,나이
customer_id,,
0,철수,40
1,영희,20
2,길동,21
3,영수,30
4,수민,31
5,동건,18


In [10]:
order1

,item,quantity
customer_id,,
1,치약,1
1,칫솔,2
2,이어폰,1
2,헤드셋,1
2,수건,3
3,생수,2
3,수건,2
1,치약,3
4,생수,2


In [11]:
# on을 명시할 필요없이 병합 가능
pd.merge(cust1, order1, left_index=True, right_index=True)

,name,나이,item,quantity
customer_id,,,,
1,영희,20,치약,1
1,영희,20,칫솔,2
1,영희,20,치약,3
2,길동,21,이어폰,1
2,길동,21,헤드셋,1
2,길동,21,수건,3
3,영수,30,생수,2
3,영수,30,수건,2
4,수민,31,생수,2


# 응용문제
1. 가장 많이 팔린 아이템은?
2. 영희가 가장 많이 구매한 아이템은?

In [12]:
# 두 테이블을 병합한 뒤 group by 함수를 활용 + sort_values로 정렬

pd.merge(customer, orders, on='customer_id').groupby('item').sum().sort_values(by='quantity', ascending=False)

,customer_id,나이,quantity
item,,,
수건,5,51,5
생수,7,61,4
치약,2,40,4
칫솔,1,20,2
이어폰,2,21,1
헤드셋,2,21,1


In [13]:
# 두 테이블을 병합한 뒤 group by + slicing을 활용

pd.merge(customer, orders, on='customer_id').groupby(['name', 'item']).sum().loc['영희', 'quantity']

item
치약    4
칫솔    2
Name: quantity, dtype: int64

# join 함수
 - 내부적으로 pandas.merge 함수 사용
 - 기본적으로 index를 사용하여 left join한다.

In [14]:
cust1.join(order1, how='inner')

,name,나이,item,quantity
customer_id,,,,
1,영희,20,치약,1
1,영희,20,칫솔,2
1,영희,20,치약,3
2,길동,21,이어폰,1
2,길동,21,헤드셋,1
2,길동,21,수건,3
3,영수,30,생수,2
3,영수,30,수건,2
4,수민,31,생수,2
